# Modelo
Para este problema de regresión se propondrán dos modelos de ensamble basados en gradient boosting: XGBoost Regressor y LightGBM, debido a su alta capacidad predictiva, rápida convergencia y buen desempeño en conjuntos de datos tabulares de tamaño medio.

Ambos modelos serán optimizados mediante GridSearchCV, con el objetivo de identificar la combinación óptima de hiperparámetros. Posteriormente, se comparará su desempeño utilizando una métrica de error adecuada para regresión y se seleccionará el modelo que minimice la función de pérdida.
 

In [8]:
import pandas as pd

X_train = pd.read_parquet("../data/X_train_prepared.parquet")
X_test  = pd.read_parquet("../data/X_test_prepared.parquet")
y_train = pd.read_parquet("../data/y_train.parquet").squeeze()
y_test  = pd.read_parquet("../data/y_test.parquet").squeeze()

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

model = xgb.XGBRegressor(random_state=42, verbosity=0)

params = {
    "n_estimators":  [300, 500,700],
    "learning_rate": [0.05, 0.1],
    "max_depth":     [8, 10, 12]
}

grid = GridSearchCV(
    model,
    params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

y_pred = grid.best_estimator_.predict(X_test)
rmse   = root_mean_squared_error(y_test, y_pred)

print(f"Mejores parámetros: {grid.best_params_}")
print(f"CV RMSE:   {-grid.best_score_:.0f}")
print(f"Test RMSE: {rmse:.0f}")

Fitting 5 folds for each of 18 candidates, totalling 90 fits


Mejores parámetros: {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 700}
CV RMSE:   46450
Test RMSE: 46509


In [11]:
from lightgbm import LGBMRegressor

lgbm_model = LGBMRegressor(random_state=42, verbose=-1)

lgbm_params = {
    "n_estimators":  [300, 500],
    "learning_rate": [0.05, 0.1],
    "num_leaves":    [31, 63, 127]   
}

lgbm_grid = GridSearchCV(
    lgbm_model,
    lgbm_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=1
)

lgbm_grid.fit(X_train, y_train)

lgbm_pred = lgbm_grid.best_estimator_.predict(X_test)
lgbm_rmse = root_mean_squared_error(y_test, lgbm_pred)

print(f"Mejores parámetros: {lgbm_grid.best_params_}")
print(f"CV RMSE:   {-lgbm_grid.best_score_:.0f}")
print(f"Test RMSE: {lgbm_rmse:.0f}")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Mejores parámetros: {'learning_rate': 0.05, 'n_estimators': 500, 'num_leaves': 63}
CV RMSE:   46104
Test RMSE: 45057


In [12]:
print(f"{'Modelo':<12} {'CV RMSE':>10} {'Test RMSE':>10}")
print("-" * 35)
print(f"{'XGBoost':<12} {-grid.best_score_:>10.0f} {rmse:>10.0f}")
print(f"{'LightGBM':<12} {-lgbm_grid.best_score_:>10.0f} {lgbm_rmse:>10.0f}")
print("-" * 35)

mejor = "XGBoost" if rmse < lgbm_rmse else "LightGBM"
print(f"\nMejor modelo: {mejor}")

Modelo          CV RMSE  Test RMSE
-----------------------------------
XGBoost           46450      46509
LightGBM          46104      45057
-----------------------------------

Mejor modelo: LightGBM
